In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import os

In [2]:
SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
data_transforms = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])

In [4]:
train_data = datasets.MNIST('data', train=True, download=True, transform=data_transforms)
test_data = datasets.MNIST('data', train=False, download=True, transform=data_transforms)

train_examples = int(len(train_data)*0.9)
valid_examples = len(train_data) - train_examples

train_data, valid_data = torch.utils.data.random_split(train_data, [train_examples, valid_examples])

In [5]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 54000
Number of validation examples: 6000
Number of testing examples: 10000


In [6]:
BATCH_SIZE = 64

train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)
test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [27]:
class MLP(nn.Module):
    def __init__(self, hidden_neurons):
        super().__init__()

        self.hidden_neurons = hidden_neurons
        
        self.input_fc = nn.Linear(28*28, hidden_neurons[0])
        self.fcs = nn.ModuleList([nn.Linear(hidden_neurons[i], hidden_neurons[i+1]) for i in range(len(hidden_neurons)-1)])
        self.output_fc = nn.Linear(hidden_neurons[-1], 10)
        
    def forward(self, x):
        
        #flatten
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.input_fc(x))
        
        for i in range(len(self.hidden_neurons)-1):
            x = F.relu(self.fcs[i](x))
        
        x = self.output_fc(x)
        
        return x

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
model = MLP([1000, 500, 250]).to(device)

In [30]:
model

MLP(
  (input_fc): Linear(in_features=784, out_features=1000, bias=True)
  (fcs): ModuleList(
    (0): Linear(in_features=1000, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=250, bias=True)
  )
  (output_fc): Linear(in_features=250, out_features=10, bias=True)
)

In [31]:
optimizer = optim.Adam(model.parameters())

In [32]:
criterion = nn.CrossEntropyLoss()

In [33]:
def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

In [34]:
def train(model, device, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        fx = model(x)
        
        loss = criterion(fx, y)
        
        acc = calculate_accuracy(fx, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [35]:
def evaluate(model, device, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            fx = model(x)

            loss = criterion(fx, y)

            acc = calculate_accuracy(fx, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [36]:
EPOCHS = 5
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'lenet-mnist.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% |')

| Epoch: 01 | Train Loss: 0.220 | Train Acc: 93.23% | Val. Loss: 0.153 | Val. Acc: 95.53% |
| Epoch: 02 | Train Loss: 0.099 | Train Acc: 97.07% | Val. Loss: 0.113 | Val. Acc: 96.71% |
| Epoch: 03 | Train Loss: 0.069 | Train Acc: 97.96% | Val. Loss: 0.103 | Val. Acc: 97.00% |
| Epoch: 04 | Train Loss: 0.057 | Train Acc: 98.33% | Val. Loss: 0.086 | Val. Acc: 97.41% |
| Epoch: 05 | Train Loss: 0.045 | Train Acc: 98.59% | Val. Loss: 0.085 | Val. Acc: 97.67% |


In [37]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

test_loss, test_acc = evaluate(model, device, valid_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}% |')

| Test Loss: 0.085 | Test Acc: 97.67% |
